# django 불러오기

In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
PWD

'c:\\Users\\my\\Documents\\GitHub\\TJ_web'

In [3]:
init_django('TJ_web') #project name

# model import

In [7]:
from steam.models import GameInfo, GameReview, com_comment

# Insert or Update

In [12]:
import pandas as pd

# game_info.csv 파일 읽기
game_info_df = pd.read_csv('game_info_final.csv')

# release_date 필드를 datetime 형식으로 변환

# GameInfo 데이터베이스에 데이터 로드
for _, row in game_info_df.iterrows():
    # release_date가 NaT인 경우, 해당 행을 건너뛰기

    game_info = GameInfo(
        title=row['game_name'], 
        genres=row['game_genres'],
        image=row['game_image'],
        price=row['game_price'],  
        release_date=row['game_release_date'],  # 날짜 형식으로 변환된 값
        mean_playtime=row['playtime_at_review'], 
    )
    game_info.save()


In [9]:
GameInfo.objects.all().delete()

(76, {'steam.GameInfo': 76})

In [ ]:
game_review_df = pd.read_csv('game_review.csv')
# GameReview 데이터베이스에 데이터 로드
for _, row in game_review_df.iterrows():
    try:
        # game_name을 통해 GameInfo에서 해당 게임 레코드 찾기
        game_info = GameInfo.objects.get(title=row['game_name'])
        
        # GameReview에 데이터 저장
        game_review = GameReview(
            game_info=game_info,  # ForeignKey로 참조하는 GameInfo 레코드
            content=row['review_text'],
            recommend=row['review_score'],
            playtime=row['playtime_at_review'],
            created_at=row['timestamp_created'],
        )
        game_review.save()
    
    except GameInfo.DoesNotExist:
        print(f"GameInfo에 해당 게임이 존재하지 않습니다: {row['game_name']}")

In [12]:
board = com_comment.objects.create(
    content = '대단한 평론가 납셧어',
    nickname = 'taeju',
    community_id = 1,
)

In [13]:
board.save()

In [14]:
board

<com_comment: com_comment object (2)>

-> 내용 확인

#### Manager 객체 - objects

In [8]:
board = Board.objects.create(
    title="게시글2",
    content="내용2",
    nickname="닉네임2"
)

In [9]:
board

<Board: Board object (2)>

In [10]:
board = Board.objects.create(
    title="게시글3",
    content="내용3",
    nickname="닉네임3"
)

In [11]:
board

<Board: Board object (3)>

### 여러 게시글을 한번에 insert하기

In [12]:
data = []
for i in range(10):
    board = Board(title=f"새로운 게시글{i}", content=f"여러게시글 insert {i}")
    data.append(board)
    


In [13]:
print(data)

[<Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>]


In [14]:
Board.objects.bulk_create(data)

[<Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>]

In [15]:
board = Board(title="게시물 추가", content="내용")
board

<Board: Board object (None)>

In [16]:
board.save()

# 조회하기

## raw query로 조회하기

In [4]:
result = Board.objects.raw('SELECT * FROM board_board')
result

<RawQuerySet: SELECT * FROM board_board>

In [5]:
for i in result[:3]:
    print(i)

Board object (1)
Board object (2)
Board object (3)


## orm으로 조회하기

### 전체 조회하기

In [14]:
board_list = community.objects.all()

In [15]:
print(board_list.query)

SELECT `steam_community`.`community_id`, `steam_community`.`title`, `steam_community`.`content`, `steam_community`.`nickname`, `steam_community`.`created_at`, `steam_community`.`updated_at`, `steam_community`.`is_deleted`, `steam_community`.`hits`, `steam_community`.`recommend` FROM `steam_community`


### filter 걸어서 조회하기(where)

In [9]:
board_list = Board.objects.filter(title="반갑")
board_list

<QuerySet [<Board: Board object (16)>]>

In [10]:
board_list = Board.objects.order_by('-created_at') # order by desc
board_list

<QuerySet [<Board: Board object (16)>, <Board: Board object (15)>, <Board: Board object (14)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>, <Board: Board object (3)>, <Board: Board object (2)>, <Board: Board object (1)>]>

In [11]:
board_list = Board.objects.order_by('-created_at')[2:4] # order by desc limit(2) offset(2)
board_list

<QuerySet [<Board: Board object (14)>, <Board: Board object (4)>]>

###  filter not 표시

In [25]:
board_list = Board.objects.exclude(title="게시물 추가")
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>]>

In [12]:
from django.db.models import Q

0001  
1001  


-> 
and: 0001  
or: 1001  
not 0001 => 1110  
xor: 1000  

####  비트연산
- ~: not
- &: and
- |: or
- ^: xor

In [27]:
board_list = Board.objects.filter(~Q(title="게시물 추가"))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>]>

## 조회하기 고급

### 다양한 조건

#### or

In [28]:
board_list = Board.objects.filter(Q(title="게시글1") | Q(title="게시글2"))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>]>

##### and 와 연산자

- gte: greater than equal
- lte: less than equal
- lt: less than
- gt: greater than

In [29]:
board_list = Board.objects.filter(Q(title__endswith="글1") & Q(id__gte=3))
board_list

<QuerySet [<Board: Board object (5)>]>

In [30]:
board_list = Board.objects.filter(Q(title__startswith="게시") & Q(created_at__year=2023))
board_list

<QuerySet []>

# 관계 추가하기

1번 댓글에 댓글을 3가지 이상 입력하세요

In [13]:
comment = Comment(
    board_id = 1,
    content = "댓글 내용2"
)
comment.save()

In [14]:
board = Board.objects.get(id=1)
comment = Comment.objects.create(
    board=board,
    content = "댓글 내용3"
)

In [33]:
for i in range(10):
    Comment.objects.create(
        board = Board.objects.get(id=2),
        content = f"댓글{i}"
    )

## 관계 조회하기

In [34]:
Comment.objects.all()

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>, <Comment: Comment object (3)>, <Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>, <Comment: Comment object (7)>, <Comment: Comment object (8)>, <Comment: Comment object (9)>, <Comment: Comment object (10)>, <Comment: Comment object (11)>, <Comment: Comment object (12)>]>

In [35]:
board_list = Board.objects.filter(id__lte=3)
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>]>

In [36]:
board = board_list[0]
board

<Board: Board object (1)>

###  board가 가지고 있는 comment 모두 조회

In [16]:
board.comment_set.all()

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>, <Comment: Comment object (14)>, <Comment: Comment object (15)>]>

In [38]:
for board in board_list:
    print(board)
    print(board.comment_set.all())
    print('-'*10)

Board object (1)
<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>]>
----------
Board object (2)
<QuerySet [<Comment: Comment object (3)>, <Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>, <Comment: Comment object (7)>, <Comment: Comment object (8)>, <Comment: Comment object (9)>, <Comment: Comment object (10)>, <Comment: Comment object (11)>, <Comment: Comment object (12)>]>
----------
Board object (3)
<QuerySet []>
----------


In [39]:
board_list = Board.objects.filter(id__lte=5).prefetch_related('comment_set')
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>]>

In [40]:
comment_list = board_list[0].comment_set.all()
comment_list

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>]>

In [41]:
comment = comment_list[0]
comment.board

<Board: Board object (1)>

# transaction

In [42]:
from django.db import transaction

In [43]:
@transaction.atomic
def sample():
    board = Board.objects.create(
        title='게시글',
        content = "atomic 게시글"
    )
    Comment.objects.create(
        board=board,
        content="atomic 댓글"
    )

In [44]:
sample()

In [45]:
with transaction.atomic():
    pass

In [46]:
file = open("sample.txt", 'wt')
file.write("파일 쓰기")
file.close()

In [47]:
with open("sample.txt", 'wt') as f:
    f.write("파일 쓰기")

### decorator - @ = 함수다

In [17]:
# func = 함수
# output = return : 함수
def new_decorator(func): # 함수를 인자로 받아서 함수를 리턴
    def wrap():
        print("decorator 시작")
        func()
        print("decorator 종료")
    return wrap

In [18]:
@new_decorator
def my_func():
    print('my_func')
my_func()

decorator 시작
my_func
decorator 종료


In [20]:
def my_func2():
    print('my_func')

In [21]:
new_decorator(my_func2)()

decorator 시작
my_func
decorator 종료


In [4]:
def func(a,b,*args,**kwargs):
  print(a)
  print(b)
  print(*args)
  print(**kwargs)

In [8]:
func(1,2,3,4,label=1,lable=2)

1
2
3 4


TypeError: 'label' is an invalid keyword argument for print()